In [1]:
import pandas as pd
import yfinance as yf

In [2]:
asset = yf.Ticker('ACA.PA').history('1y')['Close'].tz_localize(None)
benchmark = yf.Ticker('^GSPC').history('1y')['Close'].tz_localize(None)
riskfree = 0.043

In [5]:
def theta(asset: pd.Series, timeperiod: int = 252) -> float:
    """
    #### Description:
    Calculate the average return of a financial instrument over a specified time period.

    #### Parameters:
    - asset (pd.Series): Time series data representing the historical prices or returns of the financial instrument.
    - timeperiod (int, optional): The number of periods to consider for calculating the average return. Default is 252.

    #### Returns:
    - float: Average return over the specified time period.
    """
    returns = asset.pct_change().dropna()
    return (1 + returns).prod() ** (timeperiod / len(returns)) - 1

def sortino(asset: pd.Series, riskfree: float, timeperiod: int = 252) -> float:
    """
    #### Description:
    Calculate the Sortino Ratio for a financial instrument based on its historical performance.
    The Sortino Ratio measures the return per unit of downside risk, focusing only on negative volatility.

    #### Parameters:
    - asset (pd.Series): Historical price or return data of the asset.
    - riskfree (float): The risk-free rate of return, typically representing the return on a risk-free investment.
    - timeperiod (int, optional): The time period used for calculating average return and downside deviation. Default is 252.

    #### Returns:
    - float: The Sortino Ratio, a measure of risk-adjusted performance considering only downside risk.
    """
    # Calculate average return
    returns = theta(asset=asset, timeperiod=timeperiod)

    # Calculate downside deviation (only negative returns relative to risk-free rate)
    daily_returns = asset.pct_change().dropna()
    downside_returns = daily_returns[daily_returns < riskfree / timeperiod]
    downside_deviation = ( ( (downside_returns - riskfree / timeperiod) ** 2 ).mean() ** 0.5 ) * (timeperiod ** 0.5)

    # Avoid division by zero
    if downside_deviation == 0:
        return float('inf')

    # Calculate Sortino Ratio
    return (returns - riskfree) / downside_deviation

In [ ]:
def upside_capture(asset: pd.Series, benchmark: pd.Series, timeperiod: int = 252) -> float:
    """
    #### Description:
    Calculate the Upside Capture Ratio of an asset relative to a benchmark.
    The Upside Capture Ratio measures how well the asset performs relative to the benchmark
    during periods when the benchmark has positive returns.

    #### Parameters:
    - asset (pd.Series): Historical price or return data of the asset.
    - benchmark (pd.Series): Historical price or return data of the benchmark.
    - timeperiod (int, optional): The number of periods per year for annualization. Default is 252.

    #### Returns:
    - float: Upside Capture Ratio.
    """
    asset_returns = asset.pct_change().dropna()
    benchmark_returns = benchmark.pct_change().dropna()

    # Align series on dates
    asset_returns, benchmark_returns = asset_returns.align(benchmark_returns, join='inner')

    # Filter periods when benchmark return is positive
    positive_mask = benchmark_returns > 0
    asset_positive = asset_returns[positive_mask]
    benchmark_positive = benchmark_returns[positive_mask]

    if benchmark_positive.mean() == 0:
        return float('inf')

    return (asset_positive.mean() * timeperiod) / (benchmark_positive.mean() * timeperiod)


def downside_capture(asset: pd.Series, benchmark: pd.Series, timeperiod: int = 252) -> float:
    """
    #### Description:
    Calculate the Downside Capture Ratio of a financial instrument relative to a benchmark.
    The Downside Capture Ratio measures how well the asset performs relative to the benchmark
    during periods when the benchmark has negative returns.

    #### Parameters:
    - asset (pd.Series): Historical price or return data of the asset.
    - benchmark (pd.Series): Historical price or return data of the benchmark.
    - timeperiod (int, optional): The number of periods per year for annualization. Default is 252.

    #### Returns:
    - float: Downside Capture Ratio.
    """
    asset_returns = asset.pct_change().dropna()
    benchmark_returns = benchmark.pct_change().dropna()

    # Align series on dates
    asset_returns, benchmark_returns = asset_returns.align(benchmark_returns, join='inner')

    # Filter periods when benchmark return is negative
    negative_mask = benchmark_returns < 0
    asset_negative = asset_returns[negative_mask]
    benchmark_negative = benchmark_returns[negative_mask]

    if benchmark_negative.mean() == 0:
        return float('inf')

    return (asset_negative.mean() * timeperiod) / (benchmark_negative.mean() * timeperiod)


In [9]:
downside_capture(asset, benchmark)

np.float64(0.07700170555823906)